In [ ]:
#Installing necessary libraries
!pip install google-cloud-bigquery google-cloud-language requests gradio nltk

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
import vertexai
PROJECT_ID = "project-saadhna-430117" #enter your project id here
vertexai.init(project=PROJECT_ID)

In [ ]:
from google.cloud import bigquery
from google.cloud import language_v1
import requests
import datetime

client_bq = bigquery.Client(project='project-saadhna-430117')

project_id = 'project-saadhna-430117'
dataset_id = 'customer_reviews'
table_id = 'google_reviews_new'

# # Your Google Places API key
google_places_api_key = '' #enter your google places api key here

In [ ]:
# # Define the table schema
# schema = [
#     bigquery.SchemaField('review_id', bigquery.enums.SqlTypeNames.STRING),
#     bigquery.SchemaField('business_id', bigquery.enums.SqlTypeNames.STRING),
#     bigquery.SchemaField('user_id', bigquery.enums.SqlTypeNames.STRING),
#     bigquery.SchemaField('rating', bigquery.enums.SqlTypeNames.FLOAT),
#     bigquery.SchemaField('review_text', bigquery.enums.SqlTypeNames.STRING),
#     bigquery.SchemaField('created_at', bigquery.enums.SqlTypeNames.TIMESTAMP),
#     bigquery.SchemaField('sentiment_score', bigquery.enums.SqlTypeNames.FLOAT),
#     bigquery.SchemaField('sentiment_magnitude', bigquery.enums.SqlTypeNames.FLOAT),
#     bigquery.SchemaField('sentiment', bigquery.enums.SqlTypeNames.STRING)
# ]

# # Create the table
# table = bigquery.Table(f"{project_id}.{dataset_id}.{table_id}", schema=schema)
# table = client_bq.create_table(table)
# print(f"Created table {table.table_id}")

In [ ]:
def get_place_id(place_name, location):
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        'input': place_name,
        'inputtype': 'textquery',
        'fields': 'place_id',
        'locationbias': f'point:{location}',  # optional, format: "lat,lng"
        'key': google_places_api_key
    }
    response = requests.get(url, params=params)
    candidates = response.json().get('candidates', [])
    if candidates:
        return candidates[0]['place_id']
    return None

def get_google_reviews(place_id):
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=reviews&key={google_places_api_key}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if there's an HTTP error
    reviews = response.json().get('result', {}).get('reviews', [])
    return reviews

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import language_v1

# Path to your service account key file
service_account_key = '' #enter your service account key here

# Initialize the Cloud Natural Language client
client = language_v1.LanguageServiceClient.from_service_account_json(service_account_key)


In [ ]:
from google.cloud import language_v1

def analyze_sentiment(text):
    """Analyzes the sentiment of the given text."""
    client = language_v1.LanguageServiceClient()

    document = language_v1.Document(
        content=text, type_=language_v1.Document.Type.PLAIN_TEXT
    )

    sentiment = client.analyze_sentiment(request={"document": document}).document_sentiment

    return sentiment.score, sentiment.magnitude

In [ ]:
# Inserting data into BigQuery table
# from google.cloud import language_v1

# # Path to your service account key file
# service_account_key = ''  # Replace with the correct file name

# # Initialize the Cloud Natural Language client
# c = language_v1.LanguageServiceClient.from_service_account_json(service_account_key)

# # Function to analyze sentiment
# def analyze_sentiment(text):
#     # Create a document with the text content
#     document = language_v1.Document(
#         content=text,
#         type_=language_v1.Document.Type.PLAIN_TEXT
#     )

#     # Call the Cloud Natural Language API to analyze sentiment
#     sentiment = c.analyze_sentiment(request={'document': document}).document_sentiment

#     # Return sentiment score and magnitude
#     return sentiment.score, sentiment.magnitude

# def insert_reviews_to_bigquery(reviews, place_id, table_ref, p_name):
#     rows_to_insert = []
#     for review in reviews:
#         created_at = datetime.datetime.fromtimestamp(review.get('time'))
#         sentiment_score, sentiment_magnitude = analyze_sentiment(review.get('text'))

#         row = {
#             'review_id': f"{place_id}_{review.get('author_name', '')}",  # Unique ID
#             'business_id': place_id,
#             'user_id': review.get('author_name'),
#             'rating': review.get('rating'),
#             'review_text': review.get('text'),
#             'created_at': created_at.isoformat(),
#             'sentiment_score': sentiment_score,
#             'sentiment_magnitude': sentiment_magnitude,
#             'sentiment': 'positive' if sentiment_score > 0 else 'negative' if sentiment_score < 0 else 'neutral',
#             'place_name': p_name
#         }

#         # Print each row to debug
#         print("Row to insert:", row)

#         rows_to_insert.append(row)

#     if client_bq.get_table(table_ref):  # API request
#         errors = client_bq.insert_rows_json(table_ref, rows_to_insert)
#         if errors:
#             print(f'Errors occurred while inserting rows: {errors}')
#         else:
#             print('Inserted rows successfully.')
#     else:
#         print(f"Table {table_ref.table_id} doesn't exist.")

# # Example usage
# p_name = input("Enter a place: ")
# location = ''  # New York City coordinates
# place_id = get_place_id(p_name, location)

# if place_id:
#     print(f'Place ID for {p_name}: {place_id}')
#     google_reviews = get_google_reviews(place_id)
#     # Use the BigQuery client to get the table reference, including the project ID
#     table_ref = client_bq.dataset(dataset_id).table(table_id)
#     table_ref = client_bq.get_table(f"{project_id}.{dataset_id}.{table_id}")
#     insert_reviews_to_bigquery(google_reviews, place_id, table_ref, p_name)
# else:
#     print(f'Place ID for {p_name} not found.')

In [ ]:
# Creating VertexAI dataset
# import vertexai
# from google.cloud import aiplatform

# # Initialize the Vertex AI SDK
# project_id = 'project-saadhna-430117'
# region = 'us-central1'  # e.g., 'us-central1'
# aiplatform.init(project=project_id, location=region)

# # BigQuery table details
# bq_table_uri = 'bq://project-saadhna-430117.customer_reviews.google_reviews_new'

# # Create a Vertex AI Dataset
# dataset = aiplatform.TabularDataset.create(
#     display_name='customer_reviews_dataset',
#     bq_source=bq_table_uri
# )

# print(f"Created dataset: {dataset.resource_name}")

In [ ]:
# Extracting Dataset resource name
# from google.cloud import aiplatform

# # Initialize the Vertex AI client
# aiplatform.init(project='project-saadhna-430117', location='us-central1')

# # List datasets
# datasets = aiplatform.TabularDataset.list()

# # Print dataset resource names
# for dataset in datasets:
#     print(dataset.resource_name)

In [ ]:
# Training Model
# from google.cloud import aiplatform

# # Initialize the Vertex AI client
# aiplatform.init(project='project-saadhna-430117', location='us-central1')

# # Define the model training parameters
# training_task_inputs = {
#     "targetColumn": "sentiment",
#     "transformations": [
#         {"numeric": {"column_name": "sentiment_score"}},
#         {"numeric": {"column_name": "sentiment_magnitude"}},
#         {"text": {"column_name": "review_text"}}
#     ],
#     "trainBudgetMilliNodeHours": 1000,
#     "disableEarlyStopping": False
# }

# # Create and run the AutoML training job
# training_job = aiplatform.AutoMLTabularTrainingJob(
#     display_name='automl_customer_reviews_sentiment',
#     optimization_prediction_type='classification',
#     optimization_objective='minimize-log-loss'
# )

# # Replace with your actual dataset object
# dataset = aiplatform.TabularDataset('') # Replace with the actual Dataset object

# # Run the training job
# model = training_job.run(
#     dataset=dataset, # Pass the Dataset object here
#     model_display_name='customer_reviews_sentiment_model',
#     training_fraction_split=0.8,
#     validation_fraction_split=0.1,
#     test_fraction_split=0.1,
#     target_column='sentiment',
#     budget_milli_node_hours=1000,
#     disable_early_stopping=False
# )

# print(f"Trained model: {model.resource_name}")

In [ ]:
# Deploying Model to endpoint
# from google.cloud import aiplatform

# # Initialize the AI Platform client
# aiplatform.init(project='project-saadhna-430117', location='us-central1')

# # Deploy the trained model
# endpoint = model.deploy(
#     machine_type='n1-standard-4'  # Start with n1-standard-4
# )

# print(f"Deployed model to endpoint: {endpoint.resource_name}")

In [ ]:
# Testing Model
# from google.cloud import aiplatform
# from datetime import datetime

# # Initialize the Vertex AI client
# aiplatform.init(project='project-saadhna-430117', location='us-central1')
# # Specify the endpoint
# endpoint_name = 'projects/546900673488/locations/us-central1/endpoints/9122074030587772928'

# # Prepare the input data
# input_data = {
#     "instances": [
#         {
#             "review_id": "ChIJr5YkQUJiwokRxjOrpYhD6u4_S S",
#             "business_id": "ChIJr5YkQUJiwokRxjOrpYhD6u4",
#             "user_id": "S S",
#             "rating": 2.0,
#             "review_text": "So this is my first time here and I order pani puri from them it's was good but it's not worth the price for 8 pieces and the p u r I balls was very small a least you can put big one so I don't think I will ever order p a n I p u r I from them or I won't be back as a customer just a rip off. F y I I...",
#             "created_at": "2024-06-05 13:13:27UTC",  # Assuming this is the actual format
#             "sentiment_score": -0.40000000596046448,
#             "sentiment_magnitude": 0.800000011920929,
#             "place_name": "Haldirams"
#         }
#     ]
# }

# for instance in input_data['instances']:
#     # Correct the time zone offset
#     instance['created_at'] = instance['created_at'].replace("UTC", "+0000")
#     # Format the 'created_at' field
#     datetime_object = datetime.strptime(instance['created_at'], '%Y-%m-%d %H:%M:%S%z')
#     instance['created_at'] = datetime_object.strftime('%Y-%m-%d %H:%M:%S%z')

# # Create a client for the model endpoint
# endpoint = aiplatform.Endpoint(endpoint_name)

# # Make predictions
# response = endpoint.predict(instances=input_data['instances'])

# # Print the prediction results
# print(response)


In [ ]:
import os

# Ensure dependencies are installed
os.system('pip install pandas google-cloud-bigquery google-cloud-language wordcloud matplotlib nltk gradio')

# Download NLTK stopwords
import nltk
nltk.download('stopwords')

print("All dependencies are installed and NLTK stopwords are downloaded.")


In [ ]:
# !gcloud services list --enabled --project=project-saadhna-430117

In [ ]:
!gcloud auth application-default login

In [ ]:
!gcloud auth application-default set-quota-project project-saadhna-430117

In [ ]:
# Integrating Customer Sentiment Analysis output with Gradio
import pandas as pd
from google.cloud import bigquery
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import gradio as gr
import tempfile
from google.cloud import language_v1
from collections import Counter
import time

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')


# Initialize BigQuery client, explicitly providing the project ID
client = bigquery.Client(project='project-saadhna-430117')

# Initialize Google Cloud Natural Language client
language_client = language_v1.LanguageServiceClient()

# Function to fetch reviews from BigQuery
def fetch_reviews(place_name):
    query = f"""
    SELECT review_text
    FROM `project-saadhna-430117.customer_reviews.google_reviews_new`
    WHERE LOWER(place_name) = '{place_name.lower()}'
    """
    query_job = client.query(query)
    df = query_job.to_dataframe()
    return df

# Function to preprocess the text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Function to create a word cloud
def create_word_cloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    wordcloud.to_file(temp_file.name)
    return temp_file.name

# Function to analyze sentiment with retries
def analyze_sentiment(text, retries=3):
    for attempt in range(retries):
        try:
            document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
            sentiment = language_client.analyze_sentiment(request={'document': document}).document_sentiment
            return sentiment.score, sentiment.magnitude
        except Exception as e:
            if attempt < retries - 1:
                print(f"Error: {e}. Retrying...")
                time.sleep(5)
            else:
                raise

# Function to extract common keywords
def extract_keywords(text, num_keywords=10):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # POS tagging
    pos_tags = pos_tag(filtered_words)

    # Define relevant POS tags
    relevant_tags = {'NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

    # Filter words based on POS tags
    relevant_words = [word for word, tag in pos_tags if tag in relevant_tags]

    common_words = Counter(relevant_words).most_common(num_keywords)
    return [word for word, count in common_words]

# Function to generate actionable insights
def generate_insights(df):
    positive_reviews = df[df['sentiment_score'] > 0.25]['review_text']
    negative_reviews = df[df['sentiment_score'] < -0.25]['review_text']

    positive_keywords = extract_keywords(' '.join(positive_reviews))
    negative_keywords = extract_keywords(' '.join(negative_reviews))

    insights = []
    if positive_keywords:
        insights.append(f"Strengths: Common positive aspects mentioned in reviews are {', '.join(positive_keywords)}.")
    if negative_keywords:
        insights.append(f"Weaknesses: Common complaints or issues mentioned in reviews are {', '.join(negative_keywords)}.")

    overall_sentiment_score = df['sentiment_score'].mean()
    overall_sentiment_magnitude = df['sentiment_magnitude'].mean()

    if overall_sentiment_score < -0.25:
        insights.append("Consider addressing common complaints or issues mentioned in the reviews to improve customer satisfaction.")
    if overall_sentiment_score > 0.25:
        insights.append("Positive sentiment detected! Continue leveraging your strengths to maintain high customer satisfaction.")
    if overall_sentiment_magnitude > 0.5:
        insights.append("High sentiment magnitude indicates strong emotions in the reviews. Focus on areas with strong feedback.")

    return insights

# Function to handle user input and generate results
def generate_results(place_name):
    try:
        df = fetch_reviews(place_name)
        if df.empty:
            return "No reviews found for this place.", [], None, ""

        # Sentiment analysis
        df['sentiment_score'], df['sentiment_magnitude'] = zip(*df['review_text'].apply(analyze_sentiment))
        all_text = ' '.join(df['review_text'].tolist())
        insights = generate_insights(df)

        # Word cloud
        df['cleaned_text'] = df['review_text'].apply(preprocess_text)
        all_cleaned_text = ' '.join(df['cleaned_text'].tolist())
        word_cloud_image = create_word_cloud(all_cleaned_text)

        # Construct the link to the Looker Studio dashboard with a message
        dashboard_message = "<p>Click the link below to view detailed visualizations and insights in the Looker Data Studio dashboard:</p>"
        dashboard_link = f"""
        <a href="https://lookerstudio.google.com/reporting/8afc3f9a-b51b-445f-945b-8c692e573bc2?params=place_name={place_name}" target="_blank">
            Open Dashboard
        </a>
        """

        # Return results
        sentiment_summary = f"Overall Sentiment Score: {df['sentiment_score'].mean()}, Overall Sentiment Magnitude: {df['sentiment_magnitude'].mean()}"
        return sentiment_summary, insights, word_cloud_image, f"{dashboard_message}{dashboard_link}", "Created by Mihika Khemka"
    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while processing your request. Please ensure that the Cloud Natural Language API is enabled and configured correctly.", [], None, ""

# Create Gradio interface
iface = gr.Interface(
    fn=generate_results,
    inputs=gr.Textbox(lines=1, placeholder="Enter place name"),
    outputs=[
        gr.Textbox(label="Sentiment Summary"),
        gr.Textbox(label="Insights"),
        gr.Image(type="filepath", label="Word Cloud"),  # Change type to "filepath"
        gr.HTML(label="Data Studio Dashboard"),
        gr.HTML(label="Credits")
    ],
    title="Customer Sentiment Analysis and Insights",
    description="Enter the place name to get sentiment analysis of customer reviews, actionable insights, and a word cloud."
)

# Launch the Gradio app
iface.launch(debug=True)
